In [89]:
import EAC
import os 
import pandas as pd
import sqlite3

In [90]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [91]:
training = EAC.training()

In [93]:
create_meso_q = '''
CREATE TABLE upcomingTraining (
week int,
session char(20),
movement char(20),
method char(20)
)
'''

In [94]:
#each movement/lift is the pk
create_movement_table_q = '''
CREATE TABLE movements (
movement CHAR(40) PRIMARY KEY not null,
split CHAR(10),
acutalMax int,
repMaxCalc float(4),
veloEquation CHAR(30),
kilos binary
);

'''

In [145]:
#methodID is a amalgamation of method and week 
create_program_builder_q = '''
CREATE TABLE program_builder (
methodID char(30) PRIMARY KEY not null,
method char(20),
week int,
sets int,
reps int,
weight float(4),
notes char(140)
targetReps int
);
'''

In [96]:
#sessionID  = date_exercise
create_history_q = '''
CREATE TABLE history(
sessionID PRIMARY KEY not null,
sessionDate date,
session char(30),
movement CHAR(40),
method CHAR(20),
setsCompleted int,
repsCompleted int,
weightUsed int,
amrapTotal int,
newMax binary,
notes CHAR(200),
RepCalcM int);
'''

In [108]:
#veloID = date_exercise_set_rep
create_velo_q='''
CREATE TABLE velo (
movement char(40),
sessionDate date,
weight_kg float(4),
weight_lb float(4),
set_ct int,
rep int,
ecc_con char(4),
MPV_ms float(4),
peak_velo float(4),
rom_cm float(2),
rom_in float(2),
watts int,
est_max_kg float(4),
est_max_lb float(4)


)
'''

In [99]:
create_rpe_q = '''
CREATE TABLE rpe (
RPE float(1) PRIMARY KEY not null,
rep1 float(4),
rep2 float(4),
rep3 float(4),
rep4 float(4),
rep5 float(4),
rep6 float(4),
rep7 float(4),
rep8 float(4),
rep9 float(4),
rep10 float(4),
rep11 float(4),
rep12 float(4),
rep13 float(4),
rep14 float(4),
rep15 float(4),
rep16 float(4)
);'''

In [101]:

conn = sqlite3.connect('training.db')
conn.execute(create_program_builder_q)
conn.execute(create_meso_q)
conn.execute(create_rpe_q)
conn.execute(create_history_q)
conn.execute(create_movement_table_q)
conn.execute(create_velo_q)
conn.commit()


In [110]:
conn.commit()

### Data Loading

In [117]:
files  = ['./initialData/'+f for f in os.listdir('./initialData')]

In [118]:
files[0]

'./initialData/MesoLoad_06022023 - ProgramBuilder.csv'

In [125]:
df = pd.read_csv(files[0])
df = df.filter([col for col in df.columns if col.find('Unnamed')==-1])

In [127]:
df.TargetReps = df.TargetReps.fillna(0)
df.Notes = df.Notes.fillna('None')

In [128]:
df

,method,week,sets,reps,weight,Notes,TargetReps
0,5x531,1,5,5,76.5,None,9.0
1,5x531,2,5,3,81,None,7.0
2,5x531,3,5,1,85.5,None,5.0
3,wendDouble,1,3,2,76.5,None,0.0
4,wendDouble,2,3,2,81,go up at least 5% if feeling good or just repeat,0.0
...,...,...,...,...,...,...,...
86,TUTBand,1,4,"30,35,40,45",72.5,last set drop a band tension or to bdy weight...,0.0
87,contrast,1,6,"1,6,1,6,1,6","90,75,90,77.5,90,80",None,0.0
88,highWave,1,6,"7,5,3,7,5,3","75,80,82.5,+10,+10,+10",None,0.0
89,50s,1,x,50,70,as many sets to get to 50 time,0.0


In [137]:
'?'+',?'*8

'?,?,?,?,?,?,?,?,?'

In [139]:

('''INSERT into {t}
                        values ({row})'''.format(t=table,row=rowi))

'INSERT into program_builder\n                        values (?,?,?,?,?,?,?)'

In [144]:
rowi = '?'+',?'*(len(df.columns)-1)
conn.executemany('''INSERT into {t}
                        values ({row})'''.format(t=table,row=rowi),df.values)

IntegrityError: UNIQUE constraint failed: program_builder.methodID

In [143]:
rowi

'?,?,?,?,?,?,?'